In [1]:
from pathlib import Path

import dask
from dask.graph_manipulation import bind
from dask.distributed import Client

from convert_composer_to_hf import ConvertComposerToHfConfig, convert_composer_to_hf
from evaluate_model import EvaluationConfig, evaluate_model
from pipeline import PipelineConfig

# from continued_pretraining import run_continued_pretraining
from further.experiments.scripts.dispatch import launch_workflow

configs = {
    "7b_to_370m": PipelineConfig(
        from_model_name="NousResearch/Llama-2-7b-hf",
        from_model_size="7b",
        to_model_size="370m",
    ),
    "1.3b_to_370m": PipelineConfig(
        from_model_name="princeton-nlp/Sheared-LLaMA-1.3B",
        from_model_size="1.3b",
        to_model_size="370m",
    ),
}


@dask.delayed
def export_task(group: str, config: ConvertComposerToHfConfig):
    launch_workflow(convert_composer_to_hf, group=group, gpu=2, no_follow=True)(config)


@dask.delayed
def eval_task(group: str, config: EvaluationConfig):
    launch_workflow(evaluate_model, group=group, gpu=2, no_follow=True)(config)


if __name__ == "__main__":
    client = Client()
    print(client.dashboard_link)

    futures = []

    for exp_name, config in configs.items():
        assert config.to_model_dims is not None

        """Create configs"""

        # original
        original_eval_config = config.from_model_eval_config

        # pruned
        pruned_export_config = config.pruned_to_hf_config
        pruned_eval_config = config.pruning_eval_config

        # continued pretraining
        checkpoint_paths = config.continued_pretraining_config.get_checkpoint_paths()
        export_and_eval_configs = []
        for checkpoint_path in checkpoint_paths:
            export_config = ConvertComposerToHfConfig(
                model_path=str(checkpoint_path),
                model_name=checkpoint_path.name,
                network_dims=config.to_model_dims,
            )
            eval_config = EvaluationConfig(model=export_config.output_path)
            export_and_eval_configs.append((export_config, eval_config))

        """Run tasks"""

        group = f"{exp_name}-original"
        futures.append(eval_task(group, original_eval_config))

        group = f"{exp_name}-pruned"
        future = export_task(group, pruned_export_config)
        futures.append(bind(eval_task, [future])(group, pruned_eval_config))

        for export_config, eval_config in export_and_eval_configs:
            ba = config.continued_pretraining_config.get_checkpoint_id(
                export_config.model_path
            )
            group = f"{exp_name}-pretraining-ba{ba}"
            future = export_task(group, export_config)
            futures.append(bind(eval_task, [future])(group, eval_config))

    dask.compute(*futures)

http://127.0.0.1:8787/status
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hfLaunching workflow: evaluate_model

Launching workflow: convert_composer_to_hf
Launching workflow: evaluate_model
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hfLaunching workflow: convert_composer_to_hf

Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_h

/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is depr

                                                                                                                  Preparing files to send to master... 508.1KB and 86 files                                                                                                                  Preparing files to send to master... 507.4KB and 74 files                                                         
Preparing files to send to master... 508.4KB and 90 files
Preparing files to send to master... 508.4KB and 90 files
                                                         
Preparing files to send to master... 508.3KB and 89 files                                                         
                                                         
Preparing files to send to master... 508.4KB and 90 files                                                         
                                                         
Preparing files to send to master... 508.4KB and 90 filesPreparing files to send

/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/trial.py:579: FutureWarning: 'TrialOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/experiment.py:78: FutureWarning: 'ExperimentOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/model.py:186: FutureWarning: 'ModelOrderBy' is deprecated and will be removed in a future release. Please

Preparing files to send to master... 508.4KB and 90 files
Experiment started at http://mlds-determined.us.rdlabs.hpecorp.net:8080/det/experiments/13577/logs
Experiment 13547 completed with state ExperimentState.COMPLETED
Experiment 13550 completed with state ExperimentState.COMPLETED
Launching workflow: convert_composer_to_hf
Experiment 13562 completed with state ExperimentState.COMPLETED
Experiment 13554 completed with state ExperimentState.COMPLETED
Experiment 13560 completed with state ExperimentState.COMPLETED
Experiment 13557 completed with state ExperimentState.COMPLETED
Experiment 13569 completed with state ExperimentState.COMPLETED
Experiment 13567 completed with state ExperimentState.COMPLETED
Experiment 13572 completed with state ExperimentState.COMPLETED
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching

...

Experiment 13641 completed with state ExperimentState.COMPLETED
Experiment 13652 completed with state ExperimentState.COMPLETED
Experiment 13653 completed with state ExperimentState.COMPLETED
Experiment 13654 completed with state ExperimentState.COMPLETED
Experiment 13644 completed with state ExperimentState.COMPLETED


Waiting for Experiment 13597 to complete. Elapsed 4.0 minutes
Waiting for Experiment 13650 to complete. Elapsed 1.0 minutes


Experiment 13597 completed with state ExperimentState.COMPLETED


Waiting for Experiment 13612 to complete. Elapsed 4.0 minutes


Experiment 13612 completed with state ExperimentState.COMPLETED
Experiment 13650 completed with state ExperimentState.COMPLETED


In [2]:
import json
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path


# Function to extract batch number from filename
def extract_batch(filename):
    match = re.search(r"ba(\d+)-rank0", filename)
    return int(match.group(1)) if match else 0


# Function to read JSON file and extract accuracy
def read_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return {task: results["acc,none"] for task, results in data["results"].items()}


# Paths
lm_eval_dir = "/nvmefs1/daranhe/llm-shearing/out/lm_eval"
results_dir = Path("results")
results_dir.mkdir(exist_ok=True, parents=True)


# Collect data
data = []
for size in ["1.3b", "7b"]:
    # Original model
    original_file = glob.glob(
        os.path.join(
            lm_eval_dir,
            f"*{'Sheared-LLaMA-1.3B' if size == '1.3b' else 'Llama-2-7b-hf'}",
            "*.json",
        )
    )[0]
    original_data = read_json(original_file)
    for task, acc in original_data.items():
        data.append(
            {
                "size": size,
                "type": "original",
                "task": task,
                "accuracy": acc,
                "batch": 0,
            }
        )

    # Pruned model
    pruned_file = glob.glob(
        os.path.join(
            lm_eval_dir, f"*pruning_{size}_to_370m*pruned-latest-rank0.hf", "*.json"
        )
    )[0]
    pruned_data = read_json(pruned_file)
    for task, acc in pruned_data.items():
        data.append(
            {"size": size, "type": "pruned", "task": task, "accuracy": acc, "batch": 0}
        )

    # Continued pretraining checkpoints
    checkpoint_files = glob.glob(
        os.path.join(
            lm_eval_dir,
            f"*pruning_{size}_to_370m*continued_pretraining*rank0.hf",
            "*.json",
        )
    )
    for file in checkpoint_files:
        batch = extract_batch(file)
        checkpoint_data = read_json(file)
        for task, acc in checkpoint_data.items():
            data.append(
                {
                    "size": size,
                    "type": "continued_pretraining",
                    "task": task,
                    "accuracy": acc,
                    "batch": batch,
                }
            )

# Convert to DataFrame
df = pd.DataFrame(data)
df.to_csv(results_dir / "task_metrics_progression_data.csv", index=False)
df

,size,type,task,accuracy,batch
0,1.3b,original,arc_easy,0.609848,0
1,1.3b,original,piqa,0.735582,0
2,1.3b,original,sciq,0.870000,0
3,1.3b,original,winogrande,0.580900,0
4,1.3b,pruned,arc_easy,0.346801,0
...,...,...,...,...,...
219,7b,continued_pretraining,winogrande,0.524073,46800
220,7b,continued_pretraining,arc_easy,0.496633,43200
221,7b,continued_pretraining,piqa,0.653428,43200
222,7b,continued_pretraining,sciq,0.819000,43200


In [3]:
# Create plots
tasks = df["task"].unique()
for task in tasks:
    plt.figure(figsize=(12, 6))
    for size in ["1.3b", "7b"]:
        task_df = df[(df["task"] == task) & (df["size"] == size)]

        # Plot original and pruned
        sns.scatterplot(
            data=task_df[task_df["type"].isin(["original", "pruned"])],
            x="batch",
            y="accuracy",
            style="type",
            markers={"original": "o", "pruned": "*"},
            s=100,
        )

        # Plot continued pretraining
        sns.lineplot(
            data=task_df[task_df["type"] == "continued_pretraining"],
            x="batch",
            y="accuracy",
            label=f"{size} continued pretraining",
        )

    plt.ylim(0, None)
    plt.title(f"Accuracy for {task}")
    plt.xlabel("Batch")
    plt.ylabel("Accuracy")
    plt.legend(title="Model")
    plt.tight_layout()
    plt.savefig(results_dir / f"task_metrics_progression_{task}.jpg")
    plt.close()

print("Plots have been saved as JPG files.")

Plots have been saved as JPG files.
